In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import re
import pandas as pd

import requests       #to send the request to the URL
from bs4 import BeautifulSoup #to get the content in the form of HTML

# Set the path to the Microsoft Edge WebDriver executable
# Download the appropriate version from: https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
edge_driver_path = 'D:/Downloads/edgedriver_win64/msedgedriver.exe'
driver = webdriver.Edge()

In [ ]:
# Open the IMDb webpage with the list of movies
# Advanced search, filters(release date:1980-2023, title_type:Movie, lang=kn (for kannada))
url='https://www.imdb.com/search/title/?title_type=feature&release_date=1980-01-01,2023-12-01&sort=release_date,desc&num_votes=100,&primary_language=kn'
driver.get(url)

In [ ]:
s=0
npages=(3200//50)+1
# npages=1

flag=0
while s<npages and flag<50:
    try:
        button = driver.find_element(By.CLASS_NAME,'ipc-see-more__button')
        button.click()
        s+=1
        print(s)
        flag=0
        time.sleep(1)

    except Exception as e:
        flag+=1
        print(f"{flag}Error clicking the button")

In [ ]:
import requests       #to send the request to the URL
from bs4 import BeautifulSoup #to get the content in the form of HTML

page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

movie_data = soup.findAll( attrs= {'class': 'ipc-metadata-list-summary-item'})
movie_data

In [ ]:
len(movie_data)

In [ ]:
import re
import pandas as pd
movie_df=pd.DataFrame()

for e in movie_data:
    try:
        movie_name='NA'
        year='NA'
        duration='NA'
        rating='NA'
        vote_cnt='NA'
        summary='NA'
        
        original_string=e.h3.text
        split_parts = original_string.split('. ')
        movie_name = split_parts[1] #if len(split_parts) > 1 else original_string
        
        year_duration=e.find_all('span', class_='sc-1e00898e-8 hsHAHC dli-title-metadata-item')
        year=year_duration[0].text
        if len(year_duration)>1:
            duration=year_duration[1].text
        
        input_string=e.find('span', class_='ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating').text
        pattern = r'([^\xa0]+)\xa0\(([^\)]+)\)'
        match = re.search(pattern, input_string)
        if match:
            rating = match.group(1)
            vote_cnt = match.group(2)
        para=e.find('div', class_='sc-73c670dc-2 djRAjt').text
        index_of_votes = para.rfind('Votes')
        
        if index_of_votes != -1:
            summary = para[:index_of_votes]
        else:
            print("The paragraph does not contain 'Votes'.")

        temp_df={'Name':[movie_name],'Year':[year],'Duration':[duration],'Rating':[rating],'Votes':[vote_cnt],'Description':[summary]}
        temp_df=pd.DataFrame(temp_df)
        movie_df=pd.concat([movie_df,temp_df])
    except :
        print('failed')
    
movie_df

In [ ]:
movie_df['language']='Hindi'

In [ ]:
movie_df

#### Save this to csv

In [ ]:
movie_df.to_csv('kannada_movies.csv',index=False)